In [1]:
import time
import random
import numpy as np
import csv
#matrices are none other than lists of lists
matrix_A = [
    [1,2,3],
    [4,5,6],q
    [7,8,9]
]
#keeping in mind that python starts to count at 0
matrix_B = [
    [4,5,3],
    [3,4,8],
    [1,2,1]
]
for rows_a in matrix_A:#iterating through rows
    #print(rows_a)
    for element_a in rows_a:#iterating through elements
        #print(element_a)
        pass
 #matrix multiplication is row * column so I need a way to iterate through columns
# this function basically iterates through the rows 
# and takes the "Index" element of a row and puts its in a new list, which gives us a column
def matrix_column(Matrix, Index):
    column = []
    for row in Matrix:
        column.append(row[Index])
    return column
#as we can can get rows easily with a for loop, there is no reason to make a function

#now that we can iterate through rows and columns we can multiply   
def matrix_multiplication(A, B):
    result = [] #result matrix
    for row in A: #iterate through rows
        row_result = [] 
        for column_index in range(len(B[0])): #iterate through columns
            column = matrix_column(B, column_index)
            product_sum = 0
            for i in range(len(row)): #iterates through indices of the elements
                product_sum += row[i] * column[i]
            row_result.append(product_sum) # appends the sum of the products 
        result.append(row_result) # appends result rows to the matrix
    return result

def gen_matrices(size):
    matrix = []
    for k in range(size):#for columns
        row = []
        for i in range(size):
            row.append(random.randint(-99,99)) #generates rows that are "size" long
        matrix.append(row)#appends there rows to the matrix
    return matrix



with open('matrix_times.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    # header
    writer.writerow(['size', 'Own Function', 'NumPy'])
    
    
    for i in range(1, 10):
        size = 2**i
        A = gen_matrices(size)
        B = gen_matrices(size)

        start_time = time.time()
        multiplication = matrix_multiplication(A, B)
        end_time = time.time()
        computing_time = (end_time - start_time)

        np_A = np.array(A)
        np_B = np.array(B)
        start_time = time.time()
        np_multiplication = np.matmul(np_A, np_B)
        end_time = time.time()
        np_computing_time = (end_time - start_time)

        print(f"size: {size} own function: {computing_time}s , numpy: {np_computing_time}s")
        writer.writerow([size, computing_time, np_computing_time])



size: 2 own function: 5.0067901611328125e-06s , numpy: 1.3828277587890625e-05s
size: 4 own function: 9.059906005859375e-06s , numpy: 1.9073486328125e-06s
size: 8 own function: 4.410743713378906e-05s , numpy: 2.1457672119140625e-06s
size: 16 own function: 0.00028705596923828125s , numpy: 3.0994415283203125e-06s
size: 32 own function: 0.002045869827270508s , numpy: 0.00012302398681640625s
size: 64 own function: 0.01449894905090332s , numpy: 8.177757263183594e-05s
size: 128 own function: 0.11592316627502441s , numpy: 0.0006928443908691406s
size: 256 own function: 1.0033259391784668s , numpy: 0.009335041046142578s
size: 512 own function: 9.042150020599365s , numpy: 0.12659597396850586s


In [2]:
import numpy as np
import random
import time
import csv

#I will use NumPy to write code more efficient. 

def gen_matrices(size):
    return np.random.randint(-99, 99, size=(size, size))

def simple_matrix_multiplication(A, B):
    return np.dot(A, B)

#split matrix into 4
def split(matrix):
    n = matrix.shape[0]
    a = matrix[:n//2, :n//2]
    b = matrix[:n//2, n//2:]
    c = matrix[n//2:, :n//2]
    d = matrix[n//2:, n//2:]
    return a, b, c, d

def strassens_algo(A, B):
#if its 2 or smaller we can use normal matrix multiplication
    if A.shape[0] <= 2:
        return simple_matrix_multiplication(A, B)
    
    # split both matrices into 4
    a, b, c, d = split(A)
    e, f, g, h = split(B)
    
    # recursive computing
    p1 = strassens_algo(a + d, e + h)
    p2 = strassens_algo(d, g - e)
    p3 = strassens_algo(a + b, h)
    p4 = strassens_algo(b - d, g + h)
    p5 = strassens_algo(a, f - h)
    p6 = strassens_algo(c + d, e)
    p7 = strassens_algo(a - c, e + f)
    
    # combine subsubmatrices back together
    C11 = p1 + p2 - p3 + p4
    C12 = p5 + p3
    C21 = p6 + p2
    C22 = p5 + p1 - p6 - p7
    
    # combine submatrices to get the resulting matrix
    top = np.hstack((C11, C12))
    bottom = np.hstack((C21, C22))
    return np.vstack((top, bottom))

# not recursive
def strassens_no_rec(A, B):
    a, b, c, d = split(A)
    e, f, g, h = split(B)
    
    p1 = simple_matrix_multiplication(a + d, e + h)
    p2 = simple_matrix_multiplication(d, g - e)
    p3 = simple_matrix_multiplication(a + b, h)
    p4 = simple_matrix_multiplication(b - d, g + h)
    p5 = simple_matrix_multiplication(a, f - h)
    p6 = simple_matrix_multiplication(c + d, e)
    p7 = simple_matrix_multiplication(a - c, e + f)
    
    C11 = p1 + p2 - p3 + p4
    C12 = p5 + p3
    C21 = p6 + p2
    C22 = p5 + p1 - p6 - p7
    
    top = np.hstack((C11, C12))
    bottom = np.hstack((C21, C22))
    return np.vstack((top, bottom))

# Speed-Test
#I am only using 2**i because otherwise I would have to fill the matrices so they are divideble by 2 or 4
with open('matrix_times.csv', mode='a', newline='') as file:
    writer = csv.writer(file)
    
    for i in range(1, 10):  
        size = 2**i
        A = gen_matrices(size)
        B = gen_matrices(size)

        # Recursion
        start_time = time.time()
        multiplication = strassens_algo(A, B)
        end_time = time.time()
        strassens_computing_time = end_time - start_time

        # no Recursion
        start_time = time.time()
        np_multiplication = strassens_no_rec(A, B)
        end_time = time.time()
        no_rec_strassens_computing_time = end_time - start_time

        print(f"size: {size} strassens: {strassens_computing_time}s , no rec: {no_rec_strassens_computing_time}s")
        writer.writerow([size, strassens_computing_time, no_rec_strassens_computing_time])


size: 2 strassens: 0.0023109912872314453s , no rec: 0.0001888275146484375s
size: 4 strassens: 4.506111145019531e-05s , no rec: 3.314018249511719e-05s
size: 8 strassens: 0.0002868175506591797s , no rec: 4.315376281738281e-05s
size: 16 strassens: 0.001377105712890625s , no rec: 2.7179718017578125e-05s
size: 32 strassens: 0.009006977081298828s , no rec: 4.7206878662109375e-05s
size: 64 strassens: 0.047036170959472656s , no rec: 0.00010204315185546875s
size: 128 strassens: 0.2902228832244873s , no rec: 0.0006122589111328125s
size: 256 strassens: 2.089661121368408s , no rec: 0.006497859954833984s
size: 512 strassens: 14.54000735282898s , no rec: 0.07820701599121094s
